<a href="https://colab.research.google.com/github/IanHsu123/IS_4487_Tokyo/blob/main/Copy_of_day9_LiYi%20Hsu_lab_deploy_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 9 Lab, LiYi Hsu, IS 4487

What do you need to do for today's project?

1. Use the model to predict on a new dataset (without the target), then use those predictions to identify those who should be called--a contact list.
2.  Make a recommendation to the Director of Sales based on all of your analytic work for this project.

Remember that for this example we'll be using the MegaTelCo data, where the target is `leave` not `answer`.  

Note that the first set of steps below is identical to the workflow in previous labs.




#Load Libraries


In [1]:
from sklearn.tree import plot_tree
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz # Import Decision Tree Classifier
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


# Get Data

For this part of the project we will be using the model to predict whether *current* customers will churn.

Remember:  we have trained the model on historical data, which includes information about whether customers have *already* churned.  But the important use case is to predict whether *current* customers will churn.

In [3]:
# Historical data
mtc = pd.read_csv("https://raw.githubusercontent.com/jefftwebb/is_4487_base/dd870389117d5b24eee7417d5378d80496555130/Labs/DataSets/megatelco_leave_survey.csv")

# Current customer data
current_customers = pd.read_csv("https://raw.githubusercontent.com/jefftwebb/is_4487_base/main/Labs/DataSets/megatelco_new_customer_data.csv")

We should double check that this new dataset is also clean.  If it isn't there will be problems when predicting.

In [4]:
current_customers.describe()

,income,overage,leftover,house,handset_price,over_15mins_calls_per_month,average_call_duration,id
count,24.000000,24.000000,24.000000,24.000000,24.000000,24.00000,24.000000,24.000000
mean,83705.958333,110.208333,28.625000,451115.500000,385.125000,11.62500,5.125000,11275.583333
std,40593.334190,97.638632,29.793219,205207.606472,231.486184,10.66409,4.099973,4947.953779
min,20392.000000,0.000000,0.000000,173038.000000,132.000000,0.00000,1.000000,3239.000000
25%,46900.000000,0.000000,0.000000,263864.000000,203.750000,1.00000,2.000000,8620.500000
50%,90135.500000,112.000000,17.500000,443579.000000,338.000000,7.50000,4.000000,11261.500000
75%,120707.500000,197.750000,50.750000,594438.500000,416.250000,22.25000,9.000000,14307.500000
max,143929.000000,252.000000,87.000000,853464.000000,888.000000,29.00000,14.000000,19570.000000


In [5]:
current_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   college                      24 non-null     object
 1   income                       24 non-null     int64 
 2   overage                      24 non-null     int64 
 3   leftover                     24 non-null     int64 
 4   house                        24 non-null     int64 
 5   handset_price                24 non-null     int64 
 6   over_15mins_calls_per_month  24 non-null     int64 
 7   average_call_duration        24 non-null     int64 
 8   reported_satisfaction        24 non-null     object
 9   reported_usage_level         24 non-null     object
 10  considering_change_of_plan   24 non-null     object
 11  id                           24 non-null     int64 
dtypes: int64(8), object(4)
memory usage: 2.4+ KB


Looks okay.

And note that there is no target variable in the data.

# Clean data

We need to take care that we perform *exactly* the same cleaning on the new data.

Here is the cleaning/preparation for the historical MegaTelco data:


In [6]:
# Make explicit copy
mtc_clean = mtc.copy()

# filter rows
mtc_clean = mtc_clean[(mtc_clean['house'] > 0) & (mtc_clean['income'] > 0) & (mtc_clean['handset_price'] < 1000)]

# remove NAs
mtc_clean = mtc_clean.dropna()

# Recode college
mtc_clean['college'] = mtc_clean['college'].replace({'one': 'yes', 'zero': 'no'})

# change reported usage and reported satisfaction (ordered)
mtc_clean['college'] = pd.Categorical(mtc_clean['college'],
                                    ordered = False).codes

mtc_clean['considering_change_of_plan'] = pd.Categorical(mtc_clean['considering_change_of_plan'],
                                    ordered = False).codes

# change reported usage and reported satisfaction (ordered)
mtc_clean['reported_usage_level'] = pd.Categorical(mtc_clean['reported_usage_level'],
                                    categories = ['low', 'avg','high'],
                                    ordered = True).codes

mtc_clean['reported_satisfaction'] = pd.Categorical(mtc_clean['reported_satisfaction'],
                                    categories = ['low', 'avg','high'],
                                    ordered = True).codes




And here is that same cleaning applied the data on current customers:

In [7]:
# Make explicit copy: ccc refers to current customers clean
ccc = current_customers.copy()

# filter rows
ccc = ccc[(ccc['house'] > 0) & (ccc['income'] > 0) & (ccc['handset_price'] < 1000)]

# remove NAs
ccc = ccc.dropna()

# Recode college
ccc['college'] = ccc['college'].replace({'one': 'yes', 'zero': 'no'})

# change reported usage and reported satisfaction (ordered)
ccc['college'] = pd.Categorical(ccc['college'],
                                    ordered = False).codes

ccc['considering_change_of_plan'] = pd.Categorical(ccc['considering_change_of_plan'],
                                    ordered = False).codes

# change reported usage and reported satisfaction (ordered)
ccc['reported_usage_level'] = pd.Categorical(ccc['reported_usage_level'],
                                    categories = ['low', 'avg','high'],
                                    ordered = True).codes

ccc['reported_satisfaction'] = pd.Categorical(ccc['reported_satisfaction'],
                                    categories = ['low', 'avg','high'],
                                    ordered = True).codes




# Fit full model

Again, we will set `max_depth = 5` to keep the tree simple and prevent overfitting.

Since we have already determined that the model is not overfitting the data we can dispense with splitting it into train and test sets.  We will therefore use *all* the data to fit the model.

In [8]:
# split the dataframe into predictors (X) and target (y)
X = mtc_clean.drop(['id', 'leave'], axis=1)
y = mtc_clean['leave']

# initialize the tree
full_tree = DecisionTreeClassifier(criterion="entropy", max_depth = 5)

# Create Decision Tree Classifer
full_tree = full_tree.fit(X, y)

# Predict

The next step is to use the model to predict churn for the current customers.

We need to ensure that the new dataset has the same shape and data types  as the data used to fit the model.

This will entail dropping the `id` column.

In [9]:
X_new = ccc.drop(['id'], axis=1)



Now we predict the probability of churn using the new data. Remember:  we are using the model trained on the historical data, `full_model`, to predict for the new data, `ccc` (the clean current customers data).

The `predict_proba()` function returns an array with two columns that are organized according to the levels in the target:  column 0 presents the probability of `LEAVE` (the first level in the target); column 1 presents the probability of `STAY` (the second level).

Hence we need to index that array to obtain the probabilities for `LEAVE` by choosing the first column: `[:, 0]`

In [13]:
# Write your code here
full_tree.predict_proba(X_new)[:,0]

array([0.79210526, 0.5392022 , 0.79210526, 0.03333333, 0.135     ,
       0.79210526, 0.5392022 , 0.9       , 0.79210526, 0.135     ,
       0.5       , 0.79210526, 0.135     , 0.58510638, 0.79210526,
       0.135     , 0.79210526, 0.68518519, 0.58510638, 0.68518519,
       0.15821501, 0.20090293, 0.15821501, 0.79210526])

# Add predictions to the data

The next step is to append the predictions to the `current_customers` data so we can link the predictions to the customer ID.  



In [18]:
# Write your code here to create the contact list
probs = full_tree.predict_proba(X_new)[:,0]

# Add probabilities as a new column to the 'ccc' dataframe
ccc['churn_prob'] = probs

# Select only the 'id' and 'churn_prob' columns for the contact list
contact_list = ccc[['id','churn_prob']]

# Display the contact list
contact_list

,id,churn_prob
0,18429,0.792105
1,13530,0.539202
2,9171,0.792105
3,3239,0.033333
4,11815,0.135000
5,15632,0.792105
6,14127,0.539202
7,11963,0.900000
8,9013,0.792105
9,18810,0.135000


# Which customers to target for retention?

This is a contact list that can be handed off to the marketing department to direct their retention efforts!

We need to organize the list by first sorting it then filtering it to include only customers with a predicted probability of churning greater than .2.  For sorting we'll use the Pandas function, `sort_values`:

In [19]:
# Write your code here to sort the list and filter for probabilities > .2
ccc = ccc.sort_values(by='churn_prob', ascending=False)
contact_list = ccc[ccc['churn_prob'] > .2]
ccc

,college,income,overage,leftover,house,handset_price,over_15mins_calls_per_month,average_call_duration,reported_satisfaction,reported_usage_level,considering_change_of_plan,id,churn_prob
7,0,121711,181,64,468078,769,14,2,0,0,2,11963,0.900000
0,1,67983,191,0,207208,384,19,14,0,2,1,18429,0.792105
16,0,43561,230,35,482108,188,22,1,1,0,2,6514,0.792105
14,1,86658,184,12,246572,253,29,5,2,2,2,7443,0.792105
11,0,48013,197,0,222103,209,23,9,0,2,0,3244,0.792105
8,1,23071,222,87,357847,144,14,2,0,1,2,9013,0.792105
23,1,115943,200,42,173038,459,24,2,2,2,2,13861,0.792105
5,0,85716,202,0,419080,357,26,11,0,2,1,15632,0.792105
2,0,23344,252,0,285199,132,20,14,0,2,1,9171,0.792105
19,0,93699,26,56,385632,384,5,1,2,0,2,19570,0.685185


How many customer are on the contact list?

In [20]:
# Write your code here
high_churn_customers = contact_list[contact_list['churn_prob'] > 0.2]

num_high_churn_customers = high_churn_customers.shape[0]

print("Number of customers on the contact list:", num_high_churn_customers)

Number of customers on the contact list: 17
